In [17]:
import dpkt
import numpy as np

In [49]:
expm_setting = "cell-bdtraffic"
dir = "./logs/"

fn_server = dir + "uplinktest-server-" + expm_setting + ".pcap"
fn_client = dir + "uplinktest-client-" + expm_setting + ".pcap"

l2proto_server = "Ethernet"
l2proto_client = "SLL"

server_ip = b'e\x06\x1e\xb1'  # 177

num_sync_pkt = 20

In [19]:
def print_pkt(fn):
    f = open(fn, 'rb')
    packets = dpkt.pcap.Reader(f)
    i = 0
    for ts, buf in packets:

        # l2 = dpkt.sll.SLL(buf)
        l2 = dpkt.ethernet.Ethernet(buf)

        if not isinstance(l2.data, dpkt.ip.IP):
            continue
        ip = l2.data
        ipid = ip.id
        if not isinstance(ip.data, dpkt.tcp.TCP):
            continue
        tcp = ip.data
        print(
            "IP ID:{0} src:{1} dst:{2} ts:{3} tcp seq:{4} tcp flag:{5} tcp ack:{6}"
            .format(ipid, ip.src, ip.dst, ts, tcp.seq, tcp.flags, tcp.ack))

        if (ip.dst == b'e\x06\x1e\xb3'):
            print("dst {}".format(i))
        i += 1
        if i > 10:
            break


print_pkt(fn_server)

IP ID:52409 src:b'|@\x10L' dst:b'e\x06\x1e\xb1' ts:1667651408.484362 tcp seq:584227591 tcp flag:2 tcp ack:0
IP ID:0 src:b'e\x06\x1e\xb1' dst:b'|@\x10L' ts:1667651408.484396 tcp seq:2276939437 tcp flag:18 tcp ack:584227592
IP ID:52410 src:b'|@\x10L' dst:b'e\x06\x1e\xb1' ts:1667651408.535981 tcp seq:584227592 tcp flag:16 tcp ack:2276939438
IP ID:52411 src:b'|@\x10L' dst:b'e\x06\x1e\xb1' ts:1667651408.542942 tcp seq:584227592 tcp flag:24 tcp ack:2276939438
IP ID:52084 src:b'e\x06\x1e\xb1' dst:b'|@\x10L' ts:1667651408.542968 tcp seq:2276939438 tcp flag:16 tcp ack:584227629
IP ID:52085 src:b'e\x06\x1e\xb1' dst:b'|@\x10L' ts:1667651408.543012 tcp seq:2276939438 tcp flag:24 tcp ack:584227629
IP ID:52412 src:b'|@\x10L' dst:b'e\x06\x1e\xb1' ts:1667651408.592428 tcp seq:584227629 tcp flag:16 tcp ack:2276939439
IP ID:52413 src:b'|@\x10L' dst:b'e\x06\x1e\xb1' ts:1667651408.600514 tcp seq:584227629 tcp flag:24 tcp ack:2276939439
IP ID:52086 src:b'e\x06\x1e\xb1' dst:b'|@\x10L' ts:1667651408.600532 t

In [20]:
def takeId(elem):

    return elem[0]

In [36]:
def extract_pkt(fn, t_sync, l2_proto, k):

    f = open(fn, 'rb')
    packets = dpkt.pcap.Reader(f)
    pkt_in = []  # pkt into UE
    pkt_out = []  # pkt out of UE
    i = 0
    for ts, buf in packets:
        if l2_proto == "Ethernet":
            l2 = dpkt.ethernet.Ethernet(buf)
        elif l2_proto == "SLL":
            l2 = dpkt.sll.SLL(buf)
        else:
            continue

        if not isinstance(l2.data, dpkt.ip.IP):
            continue
        ip = l2.data
        if not isinstance(ip.data, dpkt.tcp.TCP):
            continue
        tcp = ip.data
        if tcp.flags % 2 == 1:  # TCP FIN packets
            continue
        tcp_seq = tcp.seq
        tcp_ack = tcp.ack

        if (ip.src == server_ip):
            pkt_in.append((tcp_ack, ts + t_sync))
        elif (ip.dst == server_ip):
            pkt_out.append((tcp_seq, ts + t_sync))

        i += 1
        if k > 0 and i >= k:
            break
        # pkt_ts = np.array(pkt_ts)
    pkt_in.sort(key=takeId)
    pkt_out.sort(key=takeId)

    f.close()
    return pkt_in, pkt_out

In [47]:
def get_clock_offset_bound(pkt_snd, pkt_rcv):
    # offset defined as the time should be added to pkt_snd timestamps
    # return the upper bound of this offset

    len_pkt_snd = len(pkt_snd)
    len_pkt_rcv = len(pkt_rcv)

    i = 0
    j = 0
    seq_r = pkt_rcv[0][0]

    send_offset_ubound = 1e6

    for i in range(len_pkt_snd):
        seq = pkt_snd[i][0]
        if j == len_pkt_rcv - 1 and seq_r < seq:
            break
        while (seq_r < seq) and (j < len_pkt_rcv - 1):
            j += 1
            seq_r = pkt_rcv[j][0]

        if not seq == seq_r:
            continue
        
        print(seq, seq_r)

        ts_snd = pkt_snd[i][1]
        ts_rcv = pkt_rcv[j][1]

        delta = ts_rcv - ts_snd
        if delta < send_offset_ubound:
            send_offset_ubound = delta

    return send_offset_ubound

In [23]:
def get_clocktime_offset(fn_server, fn_client, l2_proto_server,
                        l2_proto_client, k):

      pkts_in, pkts_out = extract_pkt(fn_server, 0.0, l2_proto_server, k)
      pktc_in, pktc_out = extract_pkt(fn_client, 0.0, l2_proto_client, k)

      server_offset_lbound = 0.0 - get_clock_offset_bound(pktc_out, pkts_out)
      server_offset_ubound = get_clock_offset_bound(pkts_in, pktc_in)

      print("t_sync lbound: ", server_offset_lbound, " t_sync ubound: ",
            server_offset_ubound, " error bound: ", abs((server_offset_ubound - server_offset_lbound)) / 2.0)

      return (server_offset_lbound + server_offset_ubound) / 2.0


In [37]:
def get_owd(pkt_snd, pkt_rcv):

    owds = []
    owd_val = []

    len_pkt_snd = len(pkt_snd)
    len_pkt_rcv = len(pkt_rcv)
    print("Total pkts snd {0} rcv {1}".format(len_pkt_snd, len_pkt_rcv))
    i = 0
    j = 0
    seq_r = pkt_rcv[0][0]

    for i in range(len_pkt_snd):
        seq = pkt_snd[i][0]
        if j == len_pkt_rcv - 1 and seq_r < seq:
            break
        while (seq_r < seq) and (j < len_pkt_rcv - 1):
            j += 1
            seq_r = pkt_rcv[j][0]
            # if j % 1000 == 0:
            #     print(i, j)
        if not seq == seq_r:
            continue

        ts_snd = pkt_snd[i][1]
        ts_rcv = pkt_rcv[j][1]
        # if i % 1000 == 0:
        #     print(i, j)

        if ts_rcv > ts_snd:
            owds.append([ts_snd, ts_rcv - ts_snd])
            owd_val.append(ts_rcv - ts_snd)
        
        else:
            pass
            # print("[time synchronization fix] ts_snd: ", ts_snd, " ts_rcv: ", ts_rcv)

    owds = np.array(owds)
    owd_val = np.array(owd_val)
    return owds, owd_val

In [25]:
def get_owd_percentile(owd, p):

    return np.percentile(owd, p)

In [26]:
def get_owd_jitter(owd):
    pass

In [ ]:
# calculate
print("Start Analysis.")

server_t_sync = get_clocktime_offset(fn_server, fn_client, l2proto_server,
                                    l2proto_client, num_sync_pkt)
print("Time synchronized.")

pkts_in, pkts_out = extract_pkt(fn_server, server_t_sync, l2proto_server, -1)
print("Finish extracting server pcap. pkts_in {0} pkts_out {1}".format(
    len(pkts_in), len(pkts_out)))
pktc_in, pktc_out = extract_pkt(fn_client, 0.0, l2proto_client, -1)
print("Finish extracting client pcap. pktc_in {0} pktc_out {1}".format(
    len(pktc_in), len(pktc_out)))

ul_owd, ul_owd_val = get_owd(pktc_out, pkts_out)
print("Finished getting uplink owd. num {0}".format(len(ul_owd_val)))
print("Uplink OWD 95ile {0} 50ile {1} avg {2}".format(
    get_owd_percentile(ul_owd_val, 95), get_owd_percentile(ul_owd_val, 50),
    np.mean(ul_owd_val)))

dl_owd, dl_owd_val = get_owd(pkts_in, pktc_in)
print("Finished getting downlink owd. num {0}".format(len(dl_owd_val)))
print("Downlink OWD 95ile {0} 50ile {1} avg {2}".format(
    get_owd_percentile(dl_owd_val, 95), get_owd_percentile(dl_owd_val, 50),
    np.mean(dl_owd_val)))


: 

In [11]:
# test get_owd
pkt_snd_eg = [(35533, 10.894779), (35554, 34.345569), (37000, 23.453356)]
pkt_rcv_eg = [(23456, 9.243940), (35554, 40.284754), (35554, 30.490589),
            (36000, 435.59596545), (37000, 45.349954), (56543, 58.545938)]
owd, _ = get_owd(pkt_snd_eg, pkt_rcv_eg)
print(owd)

Total pkts snd 3 rcv 6
[[34.345569  5.939185]
 [23.453356 21.896598]]
